In [7]:
import time
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

length = 10
width = 10 

# start_loc = [0,0]
# goal_loc = [9,9] 

# 3 Cells as holes are defined in the environment
hole = [[8,1], 
        [3,3],
        [6,6]]

# 23 Cells as wall are defined in the environment
wall = [[6,0], 
        [4,2], [5,2], [6,2], [7,2],
        [5,3], [7,3], [9,3],
        [0,4], [5,4], [9,4],
        [2,5], [5,5], [9,5],
        [2,6], 
        [2,7], [8,7], 
        [2,8], [5,8], [8,8],
        [2,9], [5,9], [8,9]]


reward_arr = np.array([[0,  -1, -1, -1, -1, -1, -1, -1, -1, -1],
                       [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
                       [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
                       [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
                       [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
                       [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
                       [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
                       [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
                       [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
                       [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1]])

actions = [[0, 1],      # Up
           [0,-1],      # Down
           [1, 0],      # Right
           [-1,0]]      # Left

poss_act = [[0, 1], [0, 1], [0, 1]]


reward_arr = np.array([])

line = []

empty_env = np.full(shape=(length,width), fill_value='.')
empty_env[0,0] = 'S'
empty_env[9,9] = 'G'

for wall_idx in wall: 
    empty_env[wall_idx[1],wall_idx[0]] = 'W' 

for hole_idx in hole: 
    empty_env[hole_idx[1],hole_idx[0]] = 'H'

for i in range(empty_env.shape[0]):
        for j in range(empty_env.shape[1]):
                line.append(empty_env[i,j])
        print(*line)
        line = []




class Env():
    def __init__(self, length, width, hole_list, wall_list, goal_loc):
        # define the width and length of the map
        self.length = length
        self.width = width

        self.hole = hole_list
        self.wall = wall_list
        # self.start_loc = start_loc   # [x_start, y_start]
        self.goal_loc = goal_loc     # [x_goal, y_goal]

        # define the agent's start position
        self.x = 0
        self.y = 0


    def render(self, frames=50):

        line = []

        empty_env = np.full(shape=(self.length,self.width), fill_value='.')
        empty_env[0,0] = 'S'
        empty_env[self.goal_loc[0],self.goal_loc[1]] = 'G'

        for wall_idx in self.wall: 
            empty_env[wall_idx[1],wall_idx[0]] = 'W' 

        for hole_idx in self.hole: 
            empty_env[hole_idx[1],hole_idx[0]] = 'H'

        for i in range(empty_env.shape[0]):
                for j in range(empty_env.shape[1]):
                        line.append(empty_env[i,j])
                print(*line)
                line = []

    def step(self, action):

        agent_state = [self.x, self.y]
        move = [[0, 1],      # Up
                [0,-1],      # Down
                [1, 0],      # Right
                [-1,0]]      # Left
        
        exception = 0

        if [agent_state[0]-1, agent_state[1]] in self.wall: 
            # Can't go left
            print("Can't go left")
            move.remove([-1,0])
            exception += 1

        if [agent_state[0]+1, agent_state[1]] in self.wall:
            # Can't go right
            print("Can't go right")
            move.remove([1,0])
            exception += 1

        if [agent_state[0], agent_state[1]+1] in self.wall:
            # Can't go up
            print("Can't go up")
            move.remove([0,1])
            exception += 1
            
        if [agent_state[0], agent_state[1]-1] in self.wall:
            # Can't go down
            print("Can't go down")
            move.remove([0,-1])
            exception += 1

        if exception != 3:
            move = move[int(random.random() * len(move))]

        # """4 legal actions, 0:up, 1:down, 2:left, 3:right"""
        # change = [[0, 1], [0, -1], [-1, 0], [1, 0]]
        # self.x = min(self.width - 1, max(0, self.x + change[action][0]))
        # self.y = min(self.length - 1, max(0, self.y + change[action][1]))

        # action = len(move)
        self.x = min(self.width - 1, max(0, self.x + move[0]))
        self.y = min(self.length - 1, max(0, self.y + move[1]))

        

        states = [self.x, self.y]
        reward = -1
        terminal = False
        if self.x == 0: # if agent is on the cliff line "SxxxxxT"
            if self.y > 0: # if agent is not on the start position 
                terminal = True
                if self.y != self.length - 1: # if agent falls
                    reward = -100
        return reward, states, terminal

    def reset(self):
        self.x = 0
        self.y = 0




class Q_table():
    def __init__(self, length, height, actions=4, alpha=0.1, gamma=0.9):
        self.table = [0] * actions * length * height # initialize all Q(s,a) to zero
        self.actions = actions
        self.length = length
        self.height = height
        self.alpha = alpha
        self.gamma = gamma

    def _index(self, a, x, y):
        """Return the index of Q([x,y], a) in Q_table."""
        return a * self.height * self.length + x * self.length + y

    def _epsilon(self):
        return 0.1
        # version for better convergence:
        # """At the beginning epsilon is 0.2, after 300 episodes decades to 0.05, and eventually go to 0."""
        # return 20. / (num_episode + 100)

    def take_action(self, x, y, num_episode):
        """epsilon-greedy action selection"""
        if random.random() < self._epsilon():
            return int(random.random() * 4)
        else:
            actions_value = [self.table[self._index(a, x, y)] for a in range(self.actions)]
            return actions_value.index(max(actions_value))

    def max_q(self, x, y):
        actions_value = [self.table[self._index(a, x, y)] for a in range(self.actions)]
        return max(actions_value)

    def update(self, a, s0, s1, r, is_terminated):
        # both s0, s1 have the form [x,y]
        q_predict = self.table[self._index(a, s0[0], s0[1])]
        if not is_terminated:
            q_target = r + self.gamma * self.max_q(s1[0], s1[1])
        else:
            q_target = r
        self.table[self._index(a, s0[0], s0[1])] += self.alpha * (q_target - q_predict)


def cliff_walk():
    env = Env(length, width, hole_list=hole, wall_list=wall, goal_loc=goal_loc)

    actions = 4

    table = Q_table(length=length, height=width, actions=actions)
    num_ep = []
    reward_ep = []
    for num_episode in range(100):
        # within the whole learning process
        episodic_reward = 0
        is_terminated = False
        s0 = [0, 0]

        while not is_terminated:
            # within one episode
            action = table.take_action(s0[0], s0[1], num_episode)
            r, s1, is_terminated = env.step(action)
            table.update(action, s0, s1, r, is_terminated)
            episodic_reward += r
            env.render(frames=1)
            s0 = s1
        if num_episode % 1 == 0:
            print("Episode: {}, Score: {}".format(num_episode, episodic_reward))
            num_ep.append(num_episode)
            reward_ep.append(episodic_reward)
            
        env.reset()

    return num_ep, reward_ep



In [11]:
length = 10
width = 10 

start_loc = [0,0]
goal_loc = [9,9] 

# 3 Cells as holes are defined in the environment
hole = [[8,1], 
        [3,3],
        [6,6]]

# 23 Cells as wall are defined in the environment
wall = [[6,0], 
        [4,2], [5,2], [6,2], [7,2],
        [5,3], [7,3], [9,3],
        [0,4], [5,4], [9,4],
        [2,5], [5,5], [9,5],
        [2,6], 
        [2,7], [8,7], 
        [2,8], [5,8], [8,8],
        [2,9], [5,9], [8,9]]

reward_arr = np.array([])

line = []

empty_env = np.full(shape=(length,width), fill_value='.')
empty_env[start_loc[0],start_loc[1]] = 'S'
empty_env[goal_loc[0],goal_loc[1]] = 'G'

for wall_idx in wall: 
    empty_env[wall_idx[1],wall_idx[0]] = 'W' 

for hole_idx in hole: 
    empty_env[hole_idx[1],hole_idx[0]] = 'H'

for i in range(empty_env.shape[0]):
        for j in range(empty_env.shape[1]):
                line.append(empty_env[i,j])
        print(*line)
        line = []



S . . . . . W . . .
. . . . . . . . H .
. . . . W W W W . .
. . . H . W . W . W
W . . . . W . . . W
. . W . . W . . . W
. . W . . . H . . .
. . W . . . . . W .
. . W . . W . . W .
. . W . . W . . W G


In [12]:

num_episode, episodic_reward = cliff_walk()

plt.figure(figsize=(10,5))
plt.plot(num_episode, episodic_reward)
plt.show()

S . . . . . W . . .
. . . . . . . . H .
. . . . W W W W . .
. . . H . W . W . W
W . . . . W . . . W
. . W . . W . . . W
. . W . . . H . . .
. . W . . . . . W .
. . W . . W . . W .
. . W . . W . . W G
S . . . . . W . . .
. . . . . . . . H .
. . . . W W W W . .
. . . H . W . W . W
W . . . . W . . . W
. . W . . W . . . W
. . W . . . H . . .
. . W . . . . . W .
. . W . . W . . W .
. . W . . W . . W G
S . . . . . W . . .
. . . . . . . . H .
. . . . W W W W . .
. . . H . W . W . W
W . . . . W . . . W
. . W . . W . . . W
. . W . . . H . . .
. . W . . . . . W .
. . W . . W . . W .
. . W . . W . . W G
S . . . . . W . . .
. . . . . . . . H .
. . . . W W W W . .
. . . H . W . W . W
W . . . . W . . . W
. . W . . W . . . W
. . W . . . H . . .
. . W . . . . . W .
. . W . . W . . W .
. . W . . W . . W G
S . . . . . W . . .
. . . . . . . . H .
. . . . W W W W . .
. . . H . W . W . W
W . . . . W . . . W
. . W . . W . . . W
. . W . . . H . . .
. . W . . . . . W .
. . W . . W . . W .
. . W . . W . . W G


TypeError: unsupported operand type(s) for +: 'int' and 'list'

In [ ]:
import random

class Env:
    def __init__(self, width, length):
        self.width = width
        self.length = length
        self.x = 0
        self.y = 0

    def step(self, action):
        move = random.choice([[0, 1], [0, -1], [-1, 0], [1, 0]])
        if isinstance(move, list):
            self.x = min(self.width - 1, max(0, self.x + move[0]))
            self.y = min(self.length - 1, max(0, self.y + move[1]))
        else:
            self.x = min(self.width - 1, max(0, self.x + move))
        states = [self.x, self.y]
        return 0, states, False

def cliff_walk():
    num_episode = [1, 2, 3, 4, 5]
    episodic_reward = [0, 1, 2, 3, 4]
    return num_episode, episodic_reward

# Test the code
num_episode, episodic_reward = cliff_walk()
print(num_episode, episodic_reward)

env = Env(10, 10)
r, s1, is_terminated = env.step(0)
print(r, s1, is_terminated)


# simple implementation

In [ ]:
length = 4
width = 4 

start_loc = [0,0]
goal_loc = [4,4] 

# 2 Cells as walls are defined in the environment
wall = [[2,2], 
        [4,2]]

# 2 Cells as holes are defined in the environment
hole = [[2,4], [3,4]]

agent_state = [2,1]

actions = [[0, 1],      # Up
           [0,-1],      # Down
           [1, 0],      # Right
           [-1,0]]      # Left

if [agent_state[0]-1, agent_state[1]] in wall: 
    # Can't go left
    print("Can't go left")
    actions.remove([-1,0])

if [agent_state[0]+1, agent_state[1]] in wall:
    # Can't go right
    print("Can't go right")
    actions.remove([1,0])

if [agent_state[0], agent_state[1]+1] in wall:
    # Can't go up
    print("Can't go up")
    actions.remove([0,1])
    
if [agent_state[0], agent_state[1]-1] in wall:
    # Can't go down
    print("Can't go down")
    actions.remove([0,-1])

print(actions)
actions = actions[int(random.random() * len(actions))]
actions

In [ ]:
(len(actions) / 4) * len(move)

In [ ]:
move = [[0, 1],      # Up
                [0,-1],      # Down
                [1, 0],      # Right
                [-1,0]]      # Left

if [agent_state[0]-1, agent_state[1]] in wall: 
            # Can't go left
            print("Can't go left")
            move.remove([-1,0])
if [agent_state[0]+1, agent_state[1]] in wall:
            # Can't go right
            print("Can't go right")
            move.remove([1,0])
if [agent_state[0], agent_state[1]+1] in wall:
            # Can't go up
            print("Can't go up")
            move.remove([0,1])

if [agent_state[0], agent_state[1]-1] in wall:
            # Can't go down
            print("Can't go down")
            move.remove([0,-1])

move = move[int(random.random() * len(move))]

        # """4 legal actions, 0:up, 1:down, 2:left, 3:right"""
        # change = [[0, 1], [0, -1], [-1, 0], [1, 0]]
        # self.x = min(self.width - 1, max(0, self.x + change[action][0]))
        # self.y = min(self.length - 1, max(0, self.y + change[action][1]))
action = len(move)
print(move)
print(action)